This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/force.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Box,
    Capsule,
    ContactResults,
    DiagramBuilder,
    InverseDynamicsController,
    JointSliders,
    LeafSystem,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    Role,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    StateInterpolatorWithDiscreteDerivative,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import StopButton
from manipulation.scenarios import AddFloatingXyzJoint, AddShape

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# A simple writing example

A popular example for class projects is to have a robot pick up a pen/pencil/chalk and write or draw something on the board. And that always begs the question "how do we make the actual ink/graphite/chalk lines appear?"

Here is a simple teleop example with a LeafSystem that will help you visualize the drawing in meshcat.  You should be able to write on basically any (one) surface at any orientation.  Go ahead and try making the chalkboard into a sphere. ;-)

You control the chalk, and see what you can draw.  
- X : arrow left/right
- Y : arrow down/up
- Z : key A / Key D

In [ ]:
# Writing System.


class MeshcatWriter(LeafSystem):
    def __init__(
        self,
        meshcat,
        drawing_geometry_ids,
        canvas_geometry_ids,
        rgba,
        line_width=0.01,
        draw_threshold=0.01,
    ):
        LeafSystem.__init__(self)
        self._meshcat = meshcat
        self._drawing_geometry_ids = drawing_geometry_ids
        self._canvas_geometry_ids = canvas_geometry_ids

        self._rgba = rgba
        self._line_width = line_width
        self._draw_threshold = draw_threshold

        self.DeclareAbstractInputPort(
            "contact_results", AbstractValue.Make(ContactResults())
        )
        self.DeclarePeriodicDiscreteUpdateEvent(0.01, 0, self.MaybeDraw)

        self._p_WLastDraw_index = self.DeclareDiscreteState(3)
        self._was_in_contact_index = self.DeclareDiscreteState([0])
        self._num_drawn_index = self.DeclareDiscreteState([0])

    def MaybeDraw(self, context, discrete_state):
        results = self.get_input_port().Eval(context)

        # The point on the canvas that most deeply penetrates the chalk.
        p_WDraw = None
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            if (
                pair.id_A in self._drawing_geometry_ids
                and pair.id_B in self._canvas_geometry_ids
            ):
                p_WDraw = pair.p_WCb
                break
            elif (
                pair.id_B in self._drawing_geometry_ids
                and pair.id_B in self._canvas_geometry_ids
            ):
                p_WDraw = pair.p_WCa
                break
        for i in range(results.num_hydroelastic_contacts()):
            surface = results.hydroelastic_contact_info(i).contact_surface()
            if (
                surface.id_M() in self._drawing_geometry_ids
                and surface.id_N() in self._canvas_geometry_ids
            ):
                p_WDraw = surface.centroid()
                break
            elif (
                surface.id_N() in self._drawing_geometry_ids
                and surface.id_M() in self._canvas_geometry_ids
            ):
                p_WDraw = surface.centroid()
                break

        if p_WDraw is not None:
            p_WLastDraw = context.get_discrete_state(
                self._p_WLastDraw_index
            ).get_value()
            was_in_contact = context.get_discrete_state(self._was_in_contact_index)[0]
            num_drawn = context.get_discrete_state(self._num_drawn_index)[0]

            length = np.linalg.norm(p_WDraw - p_WLastDraw)
            if was_in_contact and length > self._draw_threshold:
                meshcat.SetObject(
                    f"writer/{num_drawn}",
                    Capsule(self._line_width, length),
                    self._rgba,
                )
                p_WMidpoint = (p_WDraw + p_WLastDraw) / 2
                X_WCapsule = RigidTransform(
                    RotationMatrix.MakeFromOneVector(p_WDraw - p_WLastDraw, 2),
                    p_WMidpoint,
                )
                meshcat.SetTransform(f"writer/{num_drawn}", X_WCapsule)
                discrete_state.set_value(self._num_drawn_index, [num_drawn + 1])
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)
            elif not was_in_contact:
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)

            discrete_state.set_value(self._was_in_contact_index, [1])
        else:
            discrete_state.set_value(self._was_in_contact_index, [0])


def writing_example():
    builder = DiagramBuilder()

    time_step = 0.001
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    controller_plant = MultibodyPlant(time_step)
    # Add the chalk to both
    chalk_instance = AddShape(
        plant,
        Capsule(0.01, 0.2),
        "chalk",
        mass=1,
        mu=1,
        color=[1, 0.34, 0.2, 1.0],
    )
    AddFloatingXyzJoint(
        plant, plant.GetFrameByName("chalk"), chalk_instance, actuators=True
    )
    chalk_instance = AddShape(
        controller_plant,
        Capsule(0.01, 0.2),
        "chalk",
        mass=1,
        mu=1,
        color=[1, 0.34, 0.2, 1.0],
    )
    AddFloatingXyzJoint(
        controller_plant,
        controller_plant.GetFrameByName("chalk"),
        controller_plant.GetModelInstanceByName("chalk"),
        actuators=True,
    )

    # Add a writing surface to the sim plant only
    board_instance = AddShape(
        plant, Box(2, 2, 0.2), "board", color=[0.05, 0.05, 0.05, 1]
    )
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("board"),
        RigidTransform([0, 0, -0.1]),
    )

    plant.Finalize()
    controller_plant.Finalize()

    q0 = [0, 0, 0.2]
    plant.SetDefaultPositions(q0)

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    # InverseDynamicsController
    # TODO(russt): Tune the gains better
    controller = builder.AddSystem(
        InverseDynamicsController(
            controller_plant,
            kp=[20] * 3,
            ki=[0] * 3,
            kd=[10] * 3,
            has_reference_acceleration=False,
        )
    )
    builder.Connect(
        plant.get_state_output_port(chalk_instance),
        controller.get_input_port_estimated_state(),
    )
    builder.Connect(
        controller.get_output_port_control(), plant.get_actuation_input_port()
    )

    # Pose Sliders
    teleop = builder.AddSystem(
        JointSliders(
            meshcat,
            plant,
            q0,
            [-1, -1, -0.1],
            [1, 1, 0.2],
            0.02,
            decrement_keycodes=["ArrowLeft", "ArrowDown", "KeyA"],
            increment_keycodes=["ArrowRight", "ArrowUp", "KeyD"],
        )
    )

    desired_state_from_position = builder.AddSystem(
        StateInterpolatorWithDiscreteDerivative(
            3, time_step, suppress_initial_transient=True
        )
    )
    builder.Connect(
        teleop.get_output_port(), desired_state_from_position.get_input_port()
    )
    builder.Connect(
        desired_state_from_position.get_output_port(),
        controller.get_input_port_desired_state(),
    )

    inspector = scene_graph.model_inspector()
    chalk_frame_id = plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("chalk").index())
    board_frame_id = plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("board").index())
    writer = builder.AddSystem(
        MeshcatWriter(
            meshcat,
            inspector.GetGeometries(chalk_frame_id, Role.kProximity),
            inspector.GetGeometries(board_frame_id, Role.kProximity),
            Rgba(1, 0.34, 0.2, 1.0),
            line_width=0.005,
        )
    )
    builder.Connect(plant.get_contact_results_output_port(), writer.get_input_port())
    builder.AddSystem(StopButton(meshcat))

    # Simulate.
    diagram = builder.Build()

    simulator = Simulator(diagram)

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        simulator.AdvanceTo(np.inf)
    else:
        simulator.AdvanceTo(0.1)


writing_example()

In [ ]:
51 / 255

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b7f087ac-1523-4a5b-a9fa-205e230f3627' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>